<a href="https://colab.research.google.com/github/mohammadsjahanbakhsh/dim_reduction/blob/main/face_recognize_with_PCA_%26_SVC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from time import time
import seaborn as sns
import numpy as np

# Import scikit-learn library
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.svm import SVC


In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4)

In [ ]:
n_samples,h,w = lfw_people.images.shape

In [ ]:
X = lfw_people.data
X.shape

In [ ]:
imgshape=lfw_people.images.shape
## 1288 image 50*37

In [ ]:
y = lfw_people.target
target_names = lfw_people.target_names

In [ ]:
n_classes = target_names.shape[0]

In [ ]:
y.shape

In [ ]:
target_names[y[0]].split()[-1]

In [ ]:
target_names[y[0]]

In [ ]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 0,stratify=y)

In [ ]:

# Function to plot images in 3*4
def plot_gallery(images, titles, h, w, n_row = 3, n_col = 5):
    plt.figure(figsize =(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom = 0, left =.01, right =.99, top =.90, hspace =.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap = plt.cm.gray)
        plt.title(titles[i], size = 12)
        plt.xticks(())
        plt.yticks(())

# Generate true labels above the images

In [ ]:

def true_title(y,title,target_names, i):
    # true_name = target_names[y[i]].split()[-1]
    true_name = target_names[y[i]].rsplit(' ', 1)[-1]
    return '%s %s'%(title,true_name)

real_titles = [true_title(y_train,"real name:" , target_names, i)
                     for i in range(y_train.shape[0])]
plot_gallery(X_train, real_titles, h, w)

In [ ]:
t = time()
pca = PCA(n_components = 150 , svd_solver ="randomized", whiten=True)
pca.fit(X_train)

print("done in %0.3f"%(time()-t))

prinsipalfaces = pca.components_.reshape((150 , h , w))

t = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3f"%(time()-t))

In [ ]:

n_row = 4
n_col = 5
title = ["prinsipal Face: %s"%i for i in range(n_row*n_col)]

plot_gallery(prinsipalfaces, title, h,w)

In [ ]:
# GridSearch algorithm

t=time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

model = SVC(kernel ='rbf', class_weight ='balanced')
clf = GridSearchCV(
    model, param_grid ,cv=5)

clf = clf.fit(X_train_pca, y_train)
print("done in %0.3f"%(time()-t))

print(clf.best_estimator_)



In [ ]:
clf.best_estimator_

In [ ]:
names=[name.rsplit(" ",1)[-1] for name in target_names]
names

In [ ]:
t = time()
y_pred = clf.predict(X_test_pca)

print(classification_report(y_test,y_pred,target_names = names))

In [ ]:
cf_matrix = confusion_matrix(y_test,y_pred,labels = range(n_classes))
ax=sns.heatmap(cf_matrix/np.sum(cf_matrix),
            annot=True ,
            fmt='.2%',
            cmap='Blues' ,
            xticklabels = names ,
            yticklabels = names )

plt.title("Confusion Matrix",fontsize=20)
plt.xlabel('Predicted label',fontsize=15)
plt.ylabel('True label',fontsize=15)
plt.show()
# لیبل pred , ture برای محور افقی و عمودی

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cf_matrix,
                           display_labels=names)
disp.plot()
plt.show()